In [1]:
# Imports
import pinocchio as pin 
import numpy as np
from numpy.linalg import pinv, inv, norm, svd, eig
from tools import collision, getcubeplacement, setcubeplacement, projecttojointlimits
from config import LEFT_HOOK, RIGHT_HOOK, LEFT_HAND, RIGHT_HAND, EPSILON
from config import CUBE_PLACEMENT, CUBE_PLACEMENT_TARGET

pybullet build time: Sep 22 2025 15:07:41


In [2]:
# Import the implemented function from inverse_geometry.py
from inverse_geometry import computeqgrasppose

# Test it here if needed
# The actual implementation is in inverse_geometry.py as required by lab instructions

In [3]:
# Helper functions for path planning (Task 1, Part II)
# These will be useful for motion planning

def distance(q1, q2):    
    '''Return the euclidean distance between two configurations'''
    return np.linalg.norm(q2 - q1)

In [4]:
# Setup collision checking function
def coll(q):
    '''Return true if q in collision, false otherwise.'''
    pin.updateGeometryPlacements(robot.model, robot.data, robot.collision_model, robot.collision_data, q)
    return pin.computeCollisions(robot.collision_model, robot.collision_data, False)

In [ ]:
if __name__ == "__main__":
    from tools import setupwithmeshcat, setcubeplacement
    from setup_meshcat import updatevisuals
    from path import computepath, displaypath
    import time
    
    # Setup the environment
    print("Setting up robot, cube, and visualizer...")
    robot, cube, viz = setupwithmeshcat()
    
    # Start from a neutral configuration
    q = robot.q0.copy()
    
    print("\n" + "=" * 70)
    print("TASK 1 - PART I: INVERSE GEOMETRY")
    print("=" * 70)
    
    # Test 1: Compute grasp configuration at initial cube position
    print("\n[Step 1] Computing grasp pose at INITIAL cube position...")
    print(f"Cube placement: {CUBE_PLACEMENT.translation}")
    
    q0, success_init = computeqgrasppose(robot, q, cube, CUBE_PLACEMENT, viz)
    
    if success_init:
        print("✓ SUCCESS: Found collision-free grasp configuration q0")
        
        # Verify the grasp by checking hand-hook alignment
        pin.framesForwardKinematics(robot.model, robot.data, q0)
        pin.framesForwardKinematics(cube.model, cube.data, cube.q0)
        
        left_hand_id = robot.model.getFrameId(LEFT_HAND)
        right_hand_id = robot.model.getFrameId(RIGHT_HAND)
        left_hook_id = cube.model.getFrameId(LEFT_HOOK)
        right_hook_id = cube.model.getFrameId(RIGHT_HOOK)
        
        oMlhand = robot.data.oMf[left_hand_id]
        oMrhand = robot.data.oMf[right_hand_id]
        oMlhook = CUBE_PLACEMENT * cube.data.oMf[left_hook_id]
        oMrhook = CUBE_PLACEMENT * cube.data.oMf[right_hook_id]
        
        left_error = norm(pin.log(oMlhand.inverse() * oMlhook).vector)
        right_error = norm(pin.log(oMrhand.inverse() * oMrhook).vector)
        
        print(f"  Left hand alignment error: {left_error:.6f}")
        print(f"  Right hand alignment error: {right_error:.6f}")
        print(f"  Collision-free: {not collision(robot, q0)}")
        
        # Display the configuration
        setcubeplacement(robot, cube, CUBE_PLACEMENT)
        updatevisuals(viz, robot, cube, q0)
        time.sleep(1)
        
    else:
        print("✗ FAILED: Could not find collision-free grasp at initial position")
    
    # Test 2: Compute grasp configuration at target cube position
    print("\n" + "-" * 70)
    print("[Step 2] Computing grasp pose at TARGET cube position...")
    print(f"Cube placement: {CUBE_PLACEMENT_TARGET.translation}")
    
    qe, success_target = computeqgrasppose(robot, q0 if success_init else q, cube, CUBE_PLACEMENT_TARGET, viz)
    
    if success_target:
        print("✓ SUCCESS: Found collision-free grasp configuration qe")
        
        # Verify the grasp
        pin.framesForwardKinematics(robot.model, robot.data, qe)
        
        oMlhand = robot.data.oMf[left_hand_id]
        oMrhand = robot.data.oMf[right_hand_id]
        oMlhook = CUBE_PLACEMENT_TARGET * cube.data.oMf[left_hook_id]
        oMrhook = CUBE_PLACEMENT_TARGET * cube.data.oMf[right_hook_id]
        
        left_error = norm(pin.log(oMlhand.inverse() * oMlhook).vector)
        right_error = norm(pin.log(oMrhand.inverse() * oMrhook).vector)
        
        print(f"  Left hand alignment error: {left_error:.6f}")
        print(f"  Right hand alignment error: {right_error:.6f}")
        print(f"  Collision-free: {not collision(robot, qe)}")
        
        # Display the configuration
        setcubeplacement(robot, cube, CUBE_PLACEMENT_TARGET)
        updatevisuals(viz, robot, cube, qe)
        time.sleep(1)
        
    else:
        print("✗ FAILED: Could not find collision-free grasp at target position")
    
    # Summary for Part I
    print("\n" + "=" * 70)
    print("TASK 1 - PART II: MOTION PLANNING")
    print("=" * 70)
    
    if success_init and success_target:
        print("\n[Step 3] Computing collision-free path while holding cube...")
        print("Using RRT with grasping constraints")
        
        # Compute path using the implemented function
        path = computepath(q0, qe, CUBE_PLACEMENT, CUBE_PLACEMENT_TARGET)
        
        print(f"\n✓ Path computed with {len(path)} waypoints")
        
        # Display the path
        print("\n[Step 4] Displaying path...")
        print("Watch the robot move while holding the cube!")
        
        # Animate the path with cube following
        for i, q_waypoint in enumerate(path):
            viz.display(q_waypoint)
            
            # Interpolate cube placement along path
            alpha = i / (len(path) - 1) if len(path) > 1 else 1.0
            
            pos_init = CUBE_PLACEMENT.translation
            pos_target = CUBE_PLACEMENT_TARGET.translation
            pos_current = pos_init * (1 - alpha) + pos_target * alpha
            
            # Interpolate rotation
            rot_delta = CUBE_PLACEMENT.rotation.T @ CUBE_PLACEMENT_TARGET.rotation
            rot_current = CUBE_PLACEMENT.rotation @ pin.exp3(alpha * pin.log3(rot_delta))
            
            cube_placement_current = pin.SE3(rot_current, pos_current)
            setcubeplacement(robot, cube, cube_placement_current)
            
            time.sleep(0.1)  # Adjust speed as needed
        
        print("\n✓ Path execution complete!")
        
    else:
        print("\n✗ Cannot perform motion planning without valid start and goal configurations")
    
    # Final Summary
    print("\n" + "=" * 70)
    print("SUMMARY - TASK 1: GEOMETRY")
    print("=" * 70)
    print(f"Part I - Inverse Geometry:")
    print(f"  Initial grasp (q0): {'✓ Success' if success_init else '✗ Failed'}")
    print(f"  Target grasp (qe):  {'✓ Success' if success_target else '✗ Failed'}")
    print(f"\nPart II - Motion Planning:")
    if success_init and success_target:
        print(f"  Path planning:      ✓ Success ({len(path)} waypoints)")
        print(f"  Path visualization: ✓ Complete")
    else:
        print(f"  Path planning:      ✗ Skipped (missing start/goal)")
    print("\n" + "=" * 70)
    print("Task 1 (Geometry) complete!")
    print("Next: Task 2 (Dynamics) - trajectory optimization and control")
    print("=" * 70)

Setting up robot, cube, and visualizer...
Wrapper tries to connect to server <tcp://127.0.0.1:6014>
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7014/static/

TASK 1 - PART I: INVERSE GEOMETRY

[Step 1] Computing grasp pose at INITIAL cube position...
Cube placement: [ 0.33 -0.3   0.93]
✓ SUCCESS: Found collision-free grasp configuration q0
  Left hand alignment error: 0.000052
  Right hand alignment error: 0.000049
  Collision-free: True

----------------------------------------------------------------------
[Step 2] Computing grasp pose at TARGET cube position...
Cube placement: [0.4  0.11 0.93]
✓ SUCCESS: Found collision-free grasp configuration qe
  Left hand alignment error: 0.000053
  Right hand alignment error: 0.000071
  Collision-free: True

TASK 1 - PART II: MOTION PLANNING

[Step 3] Computing collision-free path while holding cube...
Using RRT with grasping constraints
Wrapper tries to connect to server <tcp://127.0.0.1:6014>
You can open the v